In [ ]:
    if __name__ == '__main__':
        %reload_ext pidgy

The shell is the application either jupyterlab or jupyter notebook, the kernel determines the programming language.  Below we design a just jupyter kernel that can be installed using 

    !pidgy kernel install

    manager = jupyter_client.kernelspec.KernelSpecManager()

    jupyter_client.kernelspec.install_kernel_spec('spec', 'pidgy')

In [2]:
    import jupyter_client

    import IPython, ipykernel, pidgy

    import traitlets

In [3]:
    class pidgyInteractiveShell(ipykernel.zmqshell.ZMQInteractiveShell):
        """Configure a native `pidgy` `IPython.InteractiveShell`"""
        enable_html_pager = traitlets.Bool(True)
        def __init__(self,*args, **kwargs):
            super().__init__(*args, **kwargs)
            pidgy.extension.load_ipython_extension(self)
    class pidgyKernel(ipykernel.ipkernel.IPythonKernel):
        shell_class = traitlets.Type(pidgyInteractiveShell)
    class pidgyKernelApp(ipykernel.kernelapp.IPKernelApp): 
        """Configure a native `pidgy` `__import__('ipykernel').kernelapp.IPKernelApp"""
        kernel_class = traitlets.Type(pidgyKernel)
        kernel = traitlets.Any(pidgyKernel)

In [4]:
    import ipykernel.kernelspec, pathlib

In [1]:
    def install(kernel_name='pidgy',
        user=True,
        replace=None,
        prefix=None
    ):
        return ipykernel.kernelspec.KernelSpecManager().install_kernel_spec(
            str(pathlib.Path(globals().get('__file__', pathlib.Path('spec'))).parent/'spec'), kernel_name=kernel_name,
            user=user, replace=replace, prefix=prefix)

In [32]:
    def uninstall(kernel_name='pidgy',):
        ipykernel.kernelspec.KernelSpecManager().remove_kernel_spec(kernel_name)

    def uninstall(kernel_name='pidgy',):
        ipykernel.kernelspec.KernelSpecManager().remove_kernel_spec(kernel_name)